In [1]:
%pip install easyocr
%pip install openai
%pip install PyPDF2
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.

In [16]:
#main libraries used, easyOCR is for text from image, openai is for GPT LLM model
#threading is for dealing with multiple files at once
#PyPDF2 is for reading text from pdf files
#docx is for dealing with word documents
import easyocr
import openai
import threading
import PyPDF2
from docx import Document

# OpenAI GPT-4 or 3.5 API credentials, enter your own because api keys are very private
openai.api_key = 'Enter API key Here'

# Function to process text from images, word files and PDF
def read_resume(file_path, output_list):
    if file_path.endswith(".pdf"):
        with open(file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page_num in range(len(pdf_reader.pages)):
                text += pdf_reader.pages[page_num].extract_text()
            output_list.append(text)
    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        output_list.append(text)
    else:
        reader = easyocr.Reader(['en'])
        result = reader.readtext(file_path)
        text = '\n'.join([x[1] for x in result])
        output_list.append(text)

# Give GPT-3.5 Turbo model the extracted texts, system is the msg from the backend to the GPT API, user is the query with the text
def analyze_resume(text,job,experience):
    response = openai.chat.completions.create	(
        model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": 'respond with only qualified for position or not qualified for position'},
                {"role": "user", "content": text+"/n"+"is this person qualified to do "+job+"that needs "+experience+"of experience ?, return qualified or not qualified "}],
        max_tokens=20)
    return (response.choices[0].message.content)

# Main method for applying
def main():
    # List of file paths (resumes), this was meant to be sent to front end via API but we had issues
    job= input("enter the job title you are hiring for ")
    experience=input("enter the needed number of years of experience ")
    file_paths = ["./screen22.png", "./ss.pdf", "./ss1.docx","./ss2.pdf"]

    output_texts = []
    threads = []

    # Using the python threading library to process multiple files
    for path in file_paths:
        thread = threading.Thread(target=read_resume, args=(path, output_texts))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    #enter each resume to a query list for the desired output of each entered resume
    for idx,text in enumerate(output_texts):
      analysis_result = (f'person{idx+1} {analyze_resume(text,job,experience)}')
      print(f"\nResume Analysis for person {idx+1}:")
      print(analysis_result)

# Apply main function
if __name__ == "__main__":
    main()



KeyboardInterrupt: Interrupted by user